In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

In [3]:
date_set_path = "../temp_sets/"

In [4]:
random_patients_df = pd.read_csv(date_set_path+"patients.csv")

In [5]:
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate
from sdv.constraints import FixedCombinations

In [6]:
model = CTGAN.load("./outputevents_model.pkl")

In [7]:
model.sample(num_rows=100)

AttributeError: 'CTGAN' object has no attribute '_model'